In [1]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.eval()

from google.colab import drive
drive.mount('/content/drive')



Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Mounted at /content/drive


In [2]:
import torch.optim as optim

def accuracy(out, labels):
  _,pred = torch.max(out, dim=1)
  return torch.sum(pred==labels).item()


num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1)

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [7]:
from torchvision.transforms.transforms import Normalize
import numpy as np
import pandas as pd
import imageio 
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import os
import albumentations as A
import cv2

preprocess=transforms.Compose([
    #transforms.Resize(256),
    transforms.CenterCrop(512),
    # transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
preprocess2 = A.Compose([
    #A.RandomCrop(width=256, height=256),
    A.HorizontalFlip(p=0.5),
    #A.RandomBrightnessContrast(p=0.2),
])

class MyDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform1=preprocess,transform2=preprocess2,dataset="tiff_wb",train=False):
        split = "train" if train else "val"
        self.brix= pd.read_csv(csv_file,sep=",")
        self.root_dir = os.path.join(root_dir, dataset, split)
        self.transform1 = transform1
        self.transform2 = transform2

        self.brix = self.brix[self.brix["split"] == split]
        self.brix = self.brix.reset_index()
        #self.img_labels = csv[["file_id", "brix", "label"]]

    def __len__(self):
        return len(self.brix)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
  
        l=self.brix.iloc[idx,2].split(".")
        name=l[0]+".tiff"
        img_name = os.path.join(self.root_dir, name)

        image = imageio.imread(img_name)
        # print("img", np.max(image))
        image = np.asarray(image, dtype=np.float32)  # convert imageio to ndarray
        image = image / (2**16-1)  # normalize between 0 and 1
        image=self.transform2(image=image)
        image = torch.as_tensor(image['image'])  # convert to tensor
        image = image.permute((2, 0, 1))  # permute the image from HWC to CHW format
        # print(image.dtype)
        # print(image.shape)
        label=np.float32(self.brix.iloc[idx,4])
        sample = {'image': image, 'label': label}
        # print("img", np.max(sample['image']))

        if self.transform1:
            sample['image'] = self.transform1(sample['image'])

        return sample

brix_dataset = MyDataset(csv_file=open('/content/drive/MyDrive/datasets/brix_labels.csv', 'r'),
                                     root_dir='drive/MyDrive/datasets')
#brix_dataset = MyDataset(csv_file=open('/home/ionut/Projects/Brix Grapes/brix_labels.csv', 'r'),
 #                                   root_dir='/home/ionut/Projects/Brix Grapes/white_balanced_images')
                                    
sample = brix_dataset[2]
input_image=sample['image']
print(sample['label'])
input_tensor=input_image
input_batch = input_tensor.unsqueeze(0)  # create a mini-batch as expected by the model

if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
    print(output)
    print(input_batch.shape)
probabilities = torch.sigmoid(output[0])
# print(input_batch)
print("Prob:", probabilities)

1.0
tensor([[0.2602]], device='cuda:0')
torch.Size([1, 3, 512, 512])
Prob: tensor([0.5647], device='cuda:0')


In [8]:
from torch.utils.data import DataLoader
from sklearn.utils import resample

#training_data,test_data=torch.utils.data.random_split(brix_dataset,[120,30])
#j=0
#for i in range(len(training_data)):
  #print(training_data[i]['image'])
 # j+=1

#print("number of samples for train:",j)
#j=0
#for i in range(len(test_data)):
  #print(training_data[i]['image'])
  #j+=1
#print("number of samples for test:",j)

training_data=brix_dataset
test_data=brix_dataset



train_dataloader = DataLoader(training_data, batch_size=6, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=6, shuffle=True)

In [5]:
data= next(iter(train_dataloader))
#print(f"Feature batch shape: {train_features.size()}")
#print(f"Labels batch shape: {train_labels.size()}")

print(data)

{'image': tensor([[[[-1.5503, -1.5343, -1.6062,  ..., -1.9101, -1.9101, -1.9421],
          [-1.5583, -1.6382, -1.5583,  ..., -1.9501, -1.9341, -1.9421],
          [-1.5822, -1.5583, -1.6142,  ..., -1.9421, -1.9581, -1.9341],
          ...,
          [-1.7981, -1.7741, -1.7821,  ..., -1.0146, -0.8307, -1.0226],
          [-1.7582, -1.7901, -1.7901,  ..., -0.6628, -0.7908, -1.0866],
          [-1.7981, -1.8301, -1.8221,  ..., -0.1192, -0.5589, -0.7908]],

         [[-1.6331, -1.6540, -1.6634,  ..., -1.8286, -1.8147, -1.8193],
          [-1.6517, -1.6424, -1.6750,  ..., -1.8333, -1.7983, -1.8495],
          [-1.6540, -1.6611, -1.6540,  ..., -1.8170, -1.8519, -1.8449],
          ...,
          [-1.6215, -1.7076, -1.7541,  ..., -0.7277, -0.7114, -0.8045],
          [-1.7029, -1.7472, -1.7472,  ..., -0.4437, -0.4390, -0.7091],
          [-1.7378, -1.7309, -1.7169,  ...,  0.0567, -0.1668, -0.4786]],

         [[-1.5312, -1.5634, -1.5794,  ..., -1.6839, -1.6839, -1.6919],
          [-1.5714, 

In [6]:
n_epochs = 15
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = 0
train_acc = []
dataset_size=MyDataset.__len__(brix_dataset)

for epoch in range(1, n_epochs+1):
  running_loss = 0.0
  correct = 0
  total=0
  print(f'Epoch {epoch}\n')
  
  for data in train_dataloader:
    data_= data['image']
    target_=data['label']
 
    target_=torch.tensor(target_).unsqueeze(0)
    if torch.cuda.is_available():
      data_, target_ = data_.to('cuda'), target_.to('cuda')
    optimizer.zero_grad()
    #data_ = data_.unsqueeze(0) 
    target_=target_.reshape([6,1])
    #print(data_.shape)     
    outputs = model(data_)
    #print(outputs,target_)
    loss = criterion(outputs, target_)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
  print(f'\ntrain-loss: {train_loss/dataset_size:.4f}')
  train_loss = 0
  # Evaluate model after each epoch

Epoch 1



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



train-loss: 0.1015
Epoch 2


train-loss: 0.0981
Epoch 3


train-loss: 0.0940
Epoch 4


train-loss: 0.0934
Epoch 5


train-loss: 0.0895
Epoch 6


train-loss: 0.0881
Epoch 7


train-loss: 0.0915
Epoch 8


train-loss: 0.0888
Epoch 9


train-loss: 0.0895
Epoch 10


train-loss: 0.0856
Epoch 11


train-loss: 0.0824
Epoch 12


train-loss: 0.0786
Epoch 13


train-loss: 0.0768
Epoch 14


train-loss: 0.0755
Epoch 15


train-loss: 0.0728


In [9]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dataloader:
        images= data['image']
        labels=data['label']
        images, labels = images.cuda(), labels.cuda()
        # calculate outputs by running images through the network
        outputs = model(images)
        probabilities = torch.sigmoid(outputs)
        #print(outputs.shape,probabilities.shape,labels.shape)
        # the class with the highest energy is what we choose as prediction
        #_, predicted = torch.max(outputs.data, 1)
        pred=torch.zeros([probabilities.shape[0]])
        print(probabilities)
        pred=pred.cuda()
        for i in range(probabilities.shape[0]):
          if(probabilities[i]>0.5):
            pred[i]=1

        #print(probabilities,labels)
        #print(labels.size(0)))
        #print(pred)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
        print(correct)

print(f'Accuracy of the network on the 10 test images: {100 * correct // total} %')

tensor([[0.4770],
        [0.5647],
        [0.8946],
        [0.8216],
        [0.9116],
        [0.9288]], device='cuda:0')
5
tensor([[0.8655],
        [0.8473],
        [0.8177],
        [0.3902]], device='cuda:0')
7
Accuracy of the network on the 10 test images: 70 %
